In [5]:
import pandas as pd

In [4]:
# Load the JSONL file
messages = pd.read_json('./data/messages/messages.json', lines=True)

# Print the first 5 rows
messages.head()

,source_id,message_id,text,impressions,reactions,shares,comments,published_at,content_type
0,253122,1415474509,https://www.youtube.com/watch?v=eR9FIPXffUw&li...,945.0,21,3.0,0,2024-06-25 11:50:25,POST
1,253122,1408345528,Плохо спится в белую петербургскую ночь. Котор...,1429.0,164,3.0,1,2024-06-24 21:32:54,POST
2,253122,1394795522,"«Давайте посмотрим внимательно, какую свободу ...",1158.0,84,9.0,1,2024-06-23 11:33:04,POST
3,253122,1364623985,К добрым словам Владимира Гельмана в мой адрес...,1071.0,86,2.0,0,2024-06-20 15:19:26,POST
4,253122,1364694325,"Известный политолог, профессор университета Хе...",1087.0,118,4.0,2,2024-06-20 15:14:28,POST


In [8]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373311 entries, 0 to 373310
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   source_id     373311 non-null  int64         
 1   message_id    373311 non-null  int64         
 2   text          373311 non-null  object        
 3   impressions   373256 non-null  float64       
 4   reactions     373311 non-null  int64         
 5   shares        370136 non-null  float64       
 6   comments      373311 non-null  int64         
 7   published_at  373311 non-null  datetime64[ns]
 8   content_type  373311 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 25.6+ MB


In [5]:
messages['content_type'].unique()

array(['POST', 'REPOST', 'COMMENT'], dtype=object)

In [7]:
messages['content_type'].value_counts()

content_type
POST       269719
REPOST     102153
COMMENT      1439
Name: count, dtype: int64

In [6]:
messages[messages['content_type']=='COMMENT']

,source_id,message_id,text,impressions,reactions,shares,comments,published_at,content_type
5988,245037,1425601312,А потом статистика что люди по возвращались у ...,0.0,0,0.0,0,2024-06-26 10:25:58,COMMENT
5989,245037,1425601307,Получается переселенцу нужно без выплат ВПО тя...,0.0,0,0.0,0,2024-06-26 10:25:15,COMMENT
5990,245037,1425601305,мабуть хочуть ще підвищити,0.0,0,0.0,0,2024-06-26 10:08:11,COMMENT
5991,245037,1425601301,"А у кого дома уже нет, тоже должны?",0.0,0,0.0,0,2024-06-26 10:08:00,COMMENT
5992,245037,1425601107,"вони повинні працювати на фронт, а не їздити ф...",0.0,0,0.0,0,2024-06-26 10:06:30,COMMENT
...,...,...,...,...,...,...,...,...,...
364267,50763,1380323556,@Antiwarcom @KyleAnzalone_ Sw...Sword fight?,43.0,2,0.0,1,2024-06-21 00:47:22,COMMENT
364271,50763,1380323553,@Antiwarcom @proud_diplomat So why does Russia...,12.0,0,0.0,0,2024-06-20 22:36:54,COMMENT
364275,50763,1380323554,@Antiwarcom @KyleAnzalone_ Have you ever heard...,48.0,1,0.0,0,2024-06-20 20:04:53,COMMENT
364276,50763,1380323555,@Antiwarcom https://t.co/lstFWFp22c,16.0,1,0.0,0,2024-06-20 20:02:04,COMMENT


Проблеми:
    - як оцінювати канал якщо повідомлення
    - різні мови (англ, укр, рос)

In [9]:
test_data = pd.read_csv('./data/test.csv')

In [10]:
test_data.head()

,source_id,source_url
0,241835,https://t.me/ViktorVikarchuk
1,228094,https://t.me/economikal
2,101992513,https://t.me/nesaharru
3,282,https://t.me/cpartisans
4,2049875,https://twitter.com/PiftonnaBonba


In [11]:
train_data = pd.read_csv('./data/train.csv')

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3260 entries, 0 to 3259
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   source_id        3260 non-null   int64 
 1   source_url       3260 non-null   object
 2   source_category  3260 non-null   object
dtypes: int64(1), object(2)
memory usage: 76.5+ KB


In [33]:
messages_with_source_url_train = pd.merge(messages, train_data, on='source_id', how='inner')
messages_with_source_url_test = pd.merge(messages, test_data, on='source_id', how='inner')


In [39]:
import re

def remove_emojis(text):
    # Define regex pattern for emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text
    return emoji_pattern.sub(r'', text)

def remove_link_mails_hashtags_tags(text):
    return ' '.join(re.sub("(\S+@\S+\.\S+)|(@[\_A-Za-z0-9]+)|(\w+:\/\/\S+)|(#\S+)", "", text).split())

def remove_card_numbers(text):
    return ' '.join(re.sub("(\d{16})", "", text).split())

def remove_phone_numbers(text):
    return ' '.join(re.sub("((\+\d{1,3})?\s?\(?\d{1,4}\)?[\s.-]?\d{3}[\s.-]?\d{4})", "", text).split())

def remove_punctuation(text):
    return ' '.join(re.sub("([^\w\s]+)|([_]+)", " ", text).split())

# def remove_english_text(text):
#     return ' '.join(re.sub("([A-Za-z]{2,})", " ", text).split())

# def remove_cut_words(text):
#     return ' '.join(re.sub("((\s|[\.]{0,})[а-яёА-яЁ]\.)", " ", text).split())

# def remove_number_abbr(text):
#     return ' '.join(re.sub("(\d+\-[а-яёА-яЁ]+)|шт", " ", text).split())

def remove_numbers(text):
    return ' '.join(re.sub("([0-9]+)", " ", text).split())

def remove_single_characters(text):
    return ' '.join(re.sub("(^|\s+)([а-яёА-яЁa-zA-Z](\s+|$))+", " ", text).split())

def to_lower(text):
    return text.lower()

def remove_template_phrases(text):
    return ' '.join(re.sub("(подпишись|подписывайтесь|подпишитесь|подписывайся|(наш\sчат)|(просим поддержать репостами)|подписаться|архангел спецназа|прислать нам|царьград|известия лднр|чат|царьград тв|ваши известия).*", " ", text).split())

def clear_text(text):
    from functools import reduce
    pipeline = [
        remove_emojis, 
        to_lower, 
        remove_link_mails_hashtags_tags, 
        remove_card_numbers, 
        remove_phone_numbers, 
        remove_punctuation, 
        remove_numbers, 
        remove_single_characters, 
        remove_template_phrases, 
        remove_punctuation
    ]

    return reduce(lambda text, func: func(text), pipeline, text)


<>:29: SyntaxWarning: invalid escape sequence '\S'
<>:32: SyntaxWarning: invalid escape sequence '\d'
<>:35: SyntaxWarning: invalid escape sequence '\+'
<>:38: SyntaxWarning: invalid escape sequence '\w'
<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:59: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\S'
<>:32: SyntaxWarning: invalid escape sequence '\d'
<>:35: SyntaxWarning: invalid escape sequence '\+'
<>:38: SyntaxWarning: invalid escape sequence '\w'
<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:59: SyntaxWarning: invalid escape sequence '\s'
C:\Users\nikol\AppData\Local\Temp\ipykernel_12476\3823187804.py:29: SyntaxWarning: invalid escape sequence '\S'
  return ' '.join(re.sub("(\S+@\S+\.\S+)|(@[\_A-Za-z0-9]+)|(\w+:\/\/\S+)|(#\S+)", "", text).split())
C:\Users\nikol\AppData\Local\Temp\ipykernel_12476\3823187804.py:32: SyntaxWarning: invalid escape sequence '\d'
  return ' '.join(re.sub("(\d{16})", "", text).split())
C:\User

In [40]:
messages_with_source_url_train["text_cleared"] = messages_with_source_url_train["text"].apply(clear_text)
messages_with_source_url_test["text_cleared"] = messages_with_source_url_test["text"].apply(clear_text)
messages_with_source_url = pd.concat([messages_with_source_url_train, messages_with_source_url_test], axis=0)

In [38]:
messages_with_source_url.to_csv('./data/messages_source_url_merged.csv', index=False)

In [ ]:
messages_with_source_url

In [6]:
messages_with_source_url = pd.read_csv('./data/messages_source_url_merged.csv')
messages_with_source_url

C:\Users\nikol\AppData\Local\Temp\ipykernel_13988\4232088883.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  messages_with_source_url = pd.read_csv('./data/messages_source_url_merged.csv')


,source_id,message_id,text,impressions,reactions,shares,comments,published_at,content_type,source_url,source_category,text_cleared
0,253091,1428333721,Порядка 150 КОИБов планирует использовать ЦИК ...,1979.0,16,2.0,0,2024-06-26 10:13:14,POST,https://t.me/cikrf,AGGRESSIVE_INFORMATION,порядка коибов планирует использовать цик на д...
1,253091,1428079971,ЦИК утвердил порядок гашения избирательных бюл...,10291.0,57,58.0,0,2024-06-26 10:00:05,POST,https://t.me/cikrf,AGGRESSIVE_INFORMATION,цик утвердил порядок гашения избирательных бюл...
2,253091,1428007536,Председатель ярославского избиркома Елена Нови...,2079.0,17,28.0,0,2024-06-26 09:30:21,POST,https://t.me/cikrf,AGGRESSIVE_INFORMATION,председатель ярославского избиркома елена нови...
3,253091,1428079972,Мандат умершего Артура Чилингарова — депутата ...,2321.0,22,4.0,0,2024-06-26 09:19:37,POST,https://t.me/cikrf,AGGRESSIVE_INFORMATION,мандат умершего артура чилингарова депутата го...
4,253091,1427996535,ЦИК обнаружил трех «экстремистов/террористов» ...,2573.0,51,23.0,0,2024-06-26 08:31:57,POST,https://t.me/cikrf,AGGRESSIVE_INFORMATION,цик обнаружил трех экстремистов террористов ср...
...,...,...,...,...,...,...,...,...,...,...,...,...
372542,226850,14247780,В Энергодар доставлена партия гуманитарной пом...,459.0,0,0.0,0,2022-06-06 08:58:36,POST,https://t.me/ochakov_ru,NaN,энергодар доставлена партия гуманитарной помощ...
372543,226850,14227471,Российские военнослужащие доставили гуманитарн...,426.0,0,0.0,0,2022-06-05 15:54:08,POST,https://t.me/ochakov_ru,NaN,российские военнослужащие доставили гуманитарн...
372544,226850,14221375,Дети из Запорожской области благодарят русских...,511.0,0,0.0,0,2022-06-05 12:26:53,POST,https://t.me/ochakov_ru,NaN,дети из запорожской области благодарят русских...
372545,226850,14213750,Волонтеры из России собирают гуманитарную помо...,407.0,0,0.0,0,2022-06-05 08:31:04,POST,https://t.me/ochakov_ru,NaN,волонтеры из россии собирают гуманитарную помо...


In [8]:
import spacy
from spacy.lang.en import stop_words as stop_words_en
from spacy.lang.uk import stop_words as stop_words_uk
from spacy.lang.ru import stop_words as stop_words_ru

stop_words = list(stop_words_ru.STOP_WORDS)#stop_words_en.STOP_WORDS | stop_words_uk.STOP_WORDS | 
stop_words

['наверняка',
 'сами',
 'вполне',
 'поколь',
 'часто',
 'должно',
 'пусть',
 'иными',
 'здесь',
 'всем',
 'нами',
 'всегда',
 'ые',
 'от',
 'буду',
 'могло',
 'ю',
 'всё',
 'наперед',
 'вся',
 'любой',
 'едят',
 'отнелиже',
 'ф',
 'эти',
 'мочь',
 'или',
 'вначале',
 'одним',
 'снова',
 'ежели',
 'хорошо',
 'явных',
 'нынешней',
 'негде',
 'сызнова',
 'ха',
 'ваше',
 'не',
 'меня',
 'данному',
 'одна',
 'далеко',
 'внакладе',
 'нужных',
 'и',
 'прочему',
 'твоих',
 'некоторых',
 'что',
 'наса',
 'л',
 'нипочем',
 'кстати',
 'паче',
 'томах',
 'смогут',
 'ему',
 'всякой',
 'находиться',
 'увы',
 'ней',
 'им',
 'где',
 'наше',
 'для',
 'ими',
 'оттуда',
 'нету',
 'такие',
 'те',
 'такая',
 'вниз',
 'давно',
 'начале',
 'своем',
 'некуда',
 'насчет',
 'зря',
 'моими',
 'прочего',
 'всех',
 'таки',
 'наверху',
 'другим',
 'вопреки',
 'оный',
 'похожем',
 'прочее',
 'бываю',
 'наш',
 'начала',
 'х',
 'сего',
 'самими',
 'наиболее',
 'ним',
 'одного',
 'на',
 'поскольку',
 'впрочем',
 'нашег

### Топ і антитоп слів

In [9]:
# from sklearn.feature_extraction.text import CountVectorizer
# import matplotlib.pyplot as plt
# from collections import Counter
# import numpy as np

# vectorizer = CountVectorizer(stop_words=stop_words)
# X = vectorizer.fit_transform(messages_with_source_url['text_cleared'])

# words = vectorizer.get_feature_names_out()
# word_counts = X.sum(axis=0)
# word_counts_dict = dict(zip(words, word_counts.tolist()[0]))

# sorted_word_counts = dict(sorted(word_counts_dict.items(), key=lambda x: x[1], reverse=False))

# N = 50

# top_words = list(sorted_word_counts.keys())[:N]
# print(f'Топ {N} слів')
# print(top_words)

ValueError: np.nan is an invalid document, expected byte or unicode string.